# 一、什么是EM算法
EM（Expectation-Maximum）算法也称期望最大化算法，曾入选“数据挖掘十大算法”中，可见EM算法在机器学习、数据挖掘中的影响力。EM算法是最常见的隐变量估计方法，在机器学习中有极为广泛的用途，例如常被用来学习高斯混合模型（Gaussian mixture model，简称GMM）的参数；隐式马尔科夫算法（HMM）、LDA主题模型的变分推断等等。

# 二、EM算法原理

EM算法是一种迭代优化策略，由于它的计算方法中每一次迭代都分两步，其中一个为期望步（E步），另一个为极大步（M步），所以算法被称为EM算法（Expectation-Maximization Algorithm）。EM算法受到缺失思想影响，最初是为了解决数据缺失情况下的参数估计问题，其算法基础和收敛有效性等问题在Dempster、Laird和Rubin三人于1977年所做的文章《Maximum likelihood from incomplete data via the EM algorithm》中给出了详细的阐述。其基本思想是：首先根据己经给出的观测数据，估计出模型参数的值；然后再依据上一步估计出的参数值估计缺失数据的值，再根据估计出的缺失数据加上之前己经观测到的数据重新再对参数值进行估计，然后反复迭代，直至最后收敛，迭代结束。

# 三、EM算法公式推导

对于n个样本观察数据$x=\left(x_{1}, x_{2}, \ldots x_{n}\right)$，找出样本的模型参数θ, 极大化模型分布的对数似然函数如下：
$$\hat{\theta}=\operatorname{argmax} \sum_{i=1}^{n} \log p\left(x_{i} ; \theta\right)$$

> **极大似然估计** \
> 假设有一枚不均匀的硬币，抛10次，8次正面，估计抛硬币出现正面的概率。\
> 凭感觉就可以知道这个概率是0.8，其实这就是极大似然的思想。下面我们来细化我们的思路。\
> 假定抛出正面的概率为$\theta$，那么抛出如题所述的这10次结果的概率为：\
> $$L(\theta)=P(8正\mid \theta)=\theta^{8} \cdot(1-\theta)^{2}$$ \
> 这个L就是似然函数。为了方便求导，我们对L取对数，得到对数似然函数LL，然后令LL的导数等于0，解得最优化的$\theta$。\
> $$L L(\theta)=\log L(\theta)=8 \log \theta+2 \log (1-\theta)$$ \
> $$\frac{d L L}{d \theta}=\frac{8}{\theta}-\frac{2}{1-\theta}=0 \Longrightarrow \theta=0.8$$ \
> 简单来说，每一组参数，都会对应一个观测结果出现的概率。极大似然估计，就是找出使得观测结果出现的概率最大的参数。

现实中$\theta$可能是多个分布的参数，也就是说这个$x$的分布并不是只属于一个分布的，而是多个分布的组合，例如混合高斯模型。如果是多个分布的参数，那么利用分别求偏导再令偏导等于0，去求参数值的方法就不好求了。我们需要引入隐含变量来简化问题，下面就是为了简化问题，而使用的方法，是别人反复研究的成果。


如果我们得到的观察数据有未观察到的隐含数据$z=\left(z_{1}, z_{2}, \ldots, z_{n}\right)$，即上文中每个样本属于哪个分布是未知的，此时我们极大化模型分布的对数似然函数如下：
$$\hat{\theta}=\operatorname{argmax} \sum_{i=1}^{n} \log p\left(x_{i} ; \theta\right)=\operatorname{argmax} \sum_{i=1}^{n} \log \sum_{z_{i}} p\left(x_{i}, z_{i} ; \theta\right)$$


上面这个式子是根据$x_i$的边缘概率计算得来，引入$z$不影响结果，但仍然没有办法直接求出θ，再进一步进行变化，
$$\sum_{i=1}^{n} \log \sum_{z_{i}} p\left(x_{i}, z_{i} ; \theta\right)=\sum_{i=1}^{n} \log \sum_{z_{i}} Q_{i}\left(z_{i}\right) \frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}$$


$Q_{i}$是$z_{i}$的概率分布函数，所以$\sum_z Q_{i} (z_{i}) = 1$。

此时我们发现，有些规律可循了，$\sum_{z_{i}} Q_{i}\left(z_{i}\right) \frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}$是个期望，因为对于某离散随机变量X来说，其数学期望$\mathrm{E}[X]=\sum_{i} p_{i} x_{i}$。竟然是个期望我们就可以利用Jensen不等式进一步往下推导。

> **Jensen不等式**\
>（1）定义 \
> 设f是定义域为实数的函数，如果对所有的实数x，f(x)的二阶导数都大于0，那么f是凸函数。\
> Jensen不等式定义如下：\
> 如果f是凸函数，X是随机变量，那么：$E[f(X)] \geq f(E[X])$。当且仅当X是常量时，该式取等号。其中，E(X)表示X的数学期望。\
> 注：Jensen不等式应用于凹函数时，不等号方向反向。当且仅当x是常量时，该不等式取等号。\
>（2）举例\
><img src="./imgs/jensen_inequation.png" alt="jensen不等式图" width="300"/> \
>图中，实线f表示凸函数，X是随机变量，有0.5的概率是a，有0.5的概率是b。X的期望值就是a和b的中值，从图中可以看到$E[f(X)] \geq f(E[X])$成立。

那么
$$
\begin{align}
X&=\frac{p\left(x_{i}, z_{i};\theta\right)}{Q_{i}\left(z_{i}\right)} \\
f(E[X])&=\sum_{i=1}^{n} \log \sum_{z_{i}} Q_{i}\left(z_{i}\right)\frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}\\
E[f(X)]&=\sum p(x)*f(x)=\sum_{i=1}^{n} \sum_{z_{i}} Q_{i}\left(z_{i}\right)\log\frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}\\
\end{align}
$$


因为f(x)在这里是log函数，以10为底的函数，它是凹函数，当jensen不等式引用于凹函数时，需要取反。

> <img src="./imgs/log.png" alt="log函数图像" width="300" />

所以
$$
\begin{align}
f(E[x])&>=E[f(x)]\\
即\sum_{i=1}^{n} \log \sum_{z_{i}} Q_{i}\left(z_{i}\right)\frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}&>=\sum_{i=1}^{n} \sum_{z_{i}} Q_{i}\left(z_{i}\right)\log\frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}\\
\end{align}
$$


到了这一步，我们发现仍然求不出最大值，这里只有个最小值。我们设前一个式子为$L(\theta)$，后一个式子为$J(z,Q)$，那么$L(\theta)>=J(z,Q)$。
那么我们可以通过不断的最大化这个下界J，来使得L(θ)不断提高，最终达到它的最大值。
<img src="imgs/em_steps.png" alt="em步骤图" width="400" />


上图首先固定θ，调整$Q(z)$使下界$J(z,Q)$上升至与$L(θ)$在此点θ处相等（绿色曲线到蓝色曲线），然后固定$Q(z)$，调整θ使下界$J(z,Q)$达到最大值（$θ_t$到$θ_{t+1}$），然后再固定θ，调整$Q(z)$……直到收敛到似然函数$L(θ)$的最大值处的θ*。这里有两个问题：什么时候下界$J(z,Q)$与$L(θ)$在此点θ处相等？为什么一定会收敛？


* **什么时候下界$𝐽(𝑧,𝑄)$与$𝐿(θ)$在此点θ处相等？**

在Jensen不等式中，当自变量X是常数的时候，等式成立。即：
$$\frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}=c, \quad c为常数 \quad \tag{1}$$


再推导下，由于$\sum_z Q_{i} z_{i} = 1$，则可以得到：分子的和等于c（分子分母都对所有$z_{i}$求和：多个等式分子分母相加不变，这个认为每个样例的两个概率比值都是c），则：
$$\sum_{z} p\left(x_{i}, z_{i} ; \theta\right)=c$$


再代入到(1)式子中，得到

$$Q_{i}\left(z_{i}\right)=\frac{p\left(x_{i}, z_{i} ; \theta\right)}{\sum_{z} p\left(x_{i}, z_{i} ; \theta\right)}=\frac{p\left(x_{i}, z_{i} ; \theta\right)}{p\left(x_{i} ; \theta\right)}=p\left(z_{i} \mid x_{i} ; \theta\right)$$

所以我们固定了$\theta$后，$Q_{i}\left(z_{i}\right)=p\left(z_{i} \mid x_{i} ; \theta\right)$时，能取到等号。也就是这个时候下界 $𝐽(𝑧,𝑄)$ 与 $𝐿(θ)$ 在此点θ处相等。


* **为什么一定会收敛？**

首先我们上面已经推出了：

$$\sum_{i=1}^{n} \log p\left(x_{i} ; \theta\right) = \sum_{i=1}^{n} \log \sum_{z_{i}} Q_{i}\left(z_{i}\right)\frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}>=\sum_{i=1}^{n} \sum_{z_{i}} Q_{i}\left(z_{i}\right)\log\frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}$$

要证明收敛性，我们只要证明：
$$\sum_{i=1}^{n} \log p\left(x_{i} ; \theta^{t+1}\right) >= \sum_{i=1}^{n} \log p\left(x_{i} ; \theta^t\right) 即可$$ 

令
$$L(\theta, \theta^{t}) = \sum\limits_{i=1}^n\sum\limits_{z^{(i)}}p( z^{(i)}|x^{(i)};\theta^{t})\log \frac{p(x^{(i)}， z^{(i)};\theta)}{p( z^{(i)}|x^{(i)};\theta^{t})} \tag{2}$$


$$H(\theta, \theta^{t}) =  \sum\limits_{i=1}^n\sum\limits_{z^{(i)}}p( z^{(i)}|x^{(i)};\theta^{t})\log \frac{p( z^{(i)}|x^{(i)};\theta)}{p( z^{(i)}|x^{(i)};\theta^{t})} \tag{3}$$


(2)-(3)得到:

$$\sum\limits_{i=1}^n \log p(x^{(i)};\theta) = L(\theta, \theta^{j}) - H(\theta, \theta^{j})$$


在上式中分别取𝜃为$𝜃^t$和$𝜃^{t+1}$，并相减得到：


$$\sum\limits_{i=1}^n \log p(x^{(i)};\theta^{t+1})  - \sum\limits_{i=1}^n \log p(x^{(i)};\theta^{t}) = [L(\theta^{t+1}, \theta^{t}) - L(\theta^{t}, \theta^{t}) ] -[H(\theta^{t+1}, \theta^{t}) - H(\theta^{t}, \theta^{t}) ]$$


要证明EM算法的收敛性，我们只需要证明上式的右边是非负的即可。

由于$𝜃^{t+1}$使得$𝐿(𝜃,𝜃^t)$极大，因此有:

$$L(\theta^{t+1}, \theta^{t}) - L(\theta^{t}, \theta^{t})  \geq 0$$

而对于第二部分：
$$
\begin{align}
H(\theta^{t+1}, \theta^{t}) - H(\theta^{t}, \theta^{t}) & = \sum\limits_{i=1}^n\sum\limits_{z^{(i)}}P( z^{(i)}|x^{(i)};\theta^{t})log\frac{P( z^{(i)}|x^{(i)};\theta^{t+1})}{P( z^{(i)}|x^{(i)};\theta^t)} \\ 
& \leq  \sum\limits_{i=1}^nlog(\sum\limits_{z^{(i)}}P( z^{(i)}|x^{(i)};\theta^{t})\frac{P( z^{(i)}|x^{(i)};\theta^{t+1})}{P( z^{(i)}|x^{(i)};\theta^t)}) 【依据jensen不等式】\\ 
& = \sum\limits_{i=1}^nlog(\sum\limits_{z^{(i)}}P( z^{(i)}|x^{(i)};\theta^{t+1})) = 0  
\end{align}
$$

至此收敛性证明完成。

从上面的推导可以看出，EM算法可以保证收敛到一个稳定点，但是却不能保证收敛到全局的极大值点，因此它是局部最优的算法，当然，如果我们的优化目标$𝐿(𝜃,𝜃^t)$是凸的，则EM算法可以保证收敛到全局最大值，这点和梯度下降法这样的迭代算法相同。

# 四、算法流程总结

**输入：**观察到的数据$x=\left(x_{1}, x_{2}, \ldots x_{n}\right)$，联合分布 $p(x, z ; \theta)$，条件分布$p(z|x ; \theta)$，最大迭代次数T。

**算法步骤：**

（1）随机初始化模型参数θ的初值 $\theta_0$ 。

（2）t=1,2,...,T 开始EM算法迭代：

E步：计算联合分布的条件概率期望：

$$Q_{i}\left(z_{i}\right)=p\left(z_{i} \mid x_{i}, \theta_{t}\right)$$

$$l\left(\theta, \theta_{t}\right)=\sum_{i=1}^{n} \sum_{z_{i}} Q_{i}\left(z_{i}\right) \log \frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}$$

M步：极大化$l\left(\theta, \theta_{t}\right)$,得到$\theta_{t+1}$:

$$\theta_{t+1}=\operatorname{argmax}l\left(\theta, \theta_{t}\right)$$

如果$\theta_{t+1}$已经收敛，则算法结束。否则继续进行E步和M步进行迭代。

<font color="red">注意：其实$\theta_{t+1}=\operatorname{argmax}l\left(\theta, \theta_{t}\right)$还可以再进行简化

简化为：
$$
\begin{align}
l\left(\theta, \theta_{t}\right)&=\operatorname{argmax}\sum_{i=1}^{n} \sum_{z_{i}} Q_{i}\left(z_{i}\right) \log \frac{p\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)} \\
&=\operatorname{argmax}\sum_{i=1}^{n}\sum_{z_{i}} \left[ Q_{i}\left(z_{i}\right) \log p\left(x_{i}, z_{i} ; \theta \right) - Q_{i}\left(z_{i}\right) \log Q_{i}\left(z_{i}\right)\right] \\
&=\operatorname{argmax}\sum_{i=1}^{n}\sum_{z_{i}}Q_{i}\left(z_{i}\right) \log p\left(x_{i}, z_{i} ; \theta \right)\\
\end{align}
$$

因为$Q_{i}\left(z_{i}\right) \log Q_{i}\left(z_{i}\right)$是常数项，去掉对于求函数最大化时的$\theta$没有影响，也可以理解为常数项求导时的导数为0。</font>


**输出：**模型参数θ。

# 五、EM实例

**三硬币问题(李航)**

假设有3枚硬币A、B、C，硬币正面出现的概率分别为$π、p、q$。进行如下抛币试验：先掷硬币A，根据其结果选择硬币B（当A正面）或硬币C（当A反面）；然后投掷选出的硬币，掷硬币的结果，出现正面记为1，出现反面记为0；独立重复进行N次试验。若N次(N=10)试验的观测结果为：


>1,1,0,1,0,0,1,0,1,1


假设只能观测到掷硬币的结果，无法观测到掷硬币的过程，问如何估计三枚硬币正面朝上的概率。


我们先来定义变量$\theta={\pi,p,q}$，套用EM算法公式：


$$Q_{i}\left(z_{i}\right)=P\left(z_{i} \mid x_{i}, \theta_{t}\right)$$

$$l\left(\theta, \theta_{t}\right)=\sum_{i=1}^{n} \sum_{z_{i}} Q_{i}\left(z_{i}\right) \log \frac{P\left(x_{i}, z_{i} ; \theta\right)}{Q_{i}\left(z_{i}\right)}$$

---
$$
\begin{align}
Q_{i}\left(z_{i}\right)&=P\left(z_{i} \mid x_{i}, \theta_{t}\right)\\
&=\frac{P\left(x_{i}, z_{i} ; \theta_t\right)}{\sum_z P\left(x_{i}, z_{i} ; \theta_t\right)}【这个计算关系上文中有】\\
&=\frac{P(x_i|z_i;\theta_t)P(z_i;\theta_t)}{\sum_z P(x_i|z_i;\theta_t)P(z_i;\theta_t)}
\end{align}
$$

因为$z_i=\{0,1\}$，所以$\sum_{z_i}Q(z_i)=Q(0)+Q(1)$，所以我们要计算出$Q(0)和Q(1)$。


$$Q_{i}\left(1\right)=\frac{P(x_i|1;\theta_t)P(1;\theta_t)}{\sum_z P(x_i|z_i;\theta_t)P(z_i;\theta_t)}$$

$$
\begin{align}
因为：P(1;\theta_t)&=\pi_t \\
P(x_i|1;\theta_t)&=p_t^{x_i} (1-p_t)^{1-x_i} \\
P(0;\theta_t)&=1-\pi_t \\
P(x_i|0;\theta_t)&=q_t^{x_i} (1-q_t)^{1-x_i} \\
所以：Q_{i}\left(1\right)&=\frac{\pi_t(p_t^{x_i} (1-p_t)^{1-x_i})}{\pi_t(p_t^{x_i} (1-p_t)^{1-x_i})+(1-\pi_t)(q_t^{x_i} (1-q_t)^{1-x_i})} \\
Q_{i}\left(0\right)&=1-Q_{i}\left(1\right) \\
\end{align}
$$

所以：
$$l(\theta,\theta_t)=\sum_{i=1}^n\left[Q_i(1)\log \frac{\pi(p^{x_i}(1-p)^{1-x_i})}{Q_i(1)} + Q_i(0)\log \frac{(1-\pi)(q^{x_i}(1-q)^{1-x_i})}{Q_i(0)}\right]$$

求每个变量的偏导：
$$
\begin{align}
\frac{\partial f}{\partial \pi}&=\sum_{i=1}^{n}[\frac{Q_i(1)}{\pi}+\frac{Q_i(1)(-1)}{1-\pi}]=\sum_{i=1}^{n} \frac{Q_i(1)-\pi}{\pi(1-\pi)} =\frac{Q_1(1)-\pi}{\pi(1-\pi)}+\frac{Q_2(1)-\pi}{\pi(1-\pi)}+...+\frac{Q_n(1)-\pi}{\pi(1-\pi)}=\frac{\sum_{i=1}^{n}Q_i-n\pi}{\pi(1-\pi)}= 0 \\
\frac{\partial f}{\partial p}&=\sum_{i=1}^{n} \frac{x_i-p}{p(1-p)} Q_i(1) = 0 \\
\frac{\partial f}{\partial q}&=\sum_{i=1}^{n} \frac{x_i-q}{q(1-q)}\left(1-Q_i(1)\right) = 0
\end{align}
$$


令偏导数为0，求得下一次变量的值：
$$
\begin{align} 
\pi &=\frac{1}{n} \sum_{i=1}^{n} Q_i(1) \\ 
p &=\frac{\sum_{i=1}^{n} Q_i(1) x_i}{\sum_{i=1}^{n} Q_i(1)} \\ 
q &=\frac{\sum_{i=1}^{n}\left(1-Q_i(1)\right) x_i}{\sum_{i=1}^{n}\left(1-Q_i(1)\right)}\\
\end
{align}$$

**代码**

In [31]:
import numpy as np

y = np.array([1,1,0,1,0,0,1,0,1,1])
N = len(y)
pi_n = 0.4
p_n = 0.6
q_n = 0.7
flag = 1
iter = 0
while flag:    
    pi = pi_n
    p = p_n
    q = q_n    
    Q = np.zeros(N)    
    i = 0
    for n in y:
        t1 = pi*np.power(p,n)*np.power(1-p,1-n)
        t2 = (1-pi)*np.power(q,n)*np.power(1-q,1-n)
        Q[i] = t1/(t1+t2)
        i = i + 1   
        
    pi_n = np.sum(Q)/N
    p_n  = np.sum(y*Q)/np.sum(Q)
    q_n  = np.sum(y*(1-Q))/np.sum(1-Q)    
    print(('%1.4f %5.4f %5.4f') % (pi_n,p_n,q_n))    
    iter = iter + 1    
    if iter==2:
        flag = 0

0.4064 0.5368 0.6432
0.4064 0.5368 0.6432


**三硬币问题**

假设有3枚硬币A、B、C，硬币正面出现的概率分别为$π、p、q$。进行如下抛币试验：先掷硬币A，根据其结果选择硬币B（当A正面）或硬币C（当A反面）；然后投掷选出的硬币抛掷N次(N=10)，掷硬币的结果，出现正面记为1，出现反面记为0，，独立重复进行M次(M=5)试验；试验的观测结果为：

>1,0,0,0,1,1,0,1,0,1 \
>1,1,1,1,0,1,1,1,1,1 \
>1,0,1,1,1,1,1,0,1,1 \
>1,0,1,0,0,0,1,1,0,0 \
>0,1,1,1,0,1,1,1,0,1 

假设只能观测到掷硬币的结果，无法观测到掷硬币的过程，问如何估计三枚硬币正面朝上的概率。

<font color="red">实质：相比于第一个三硬币问题(李航)，这个三硬币问题就相当于抛掷完A后，再抛掷B或C时，多抛掷几次，概率多乘几次而已。</font>

我们先来定义变量$\theta={\pi,p,q}$，套用EM算法公式：


$$Q_{j}\left(z_{j}\right)=p\left(z_{j} \mid x_{ji}, \theta_{t}\right)$$

$$l\left(\theta, \theta_{t}\right)=\sum_{j=1}^m\sum_{i=1}^{n} \sum_{z_{j}} Q_{j}\left(z_{j}\right) \log \frac{p\left(x_{ji}, z_{j} ; \theta\right)}{Q_{j}\left(z_{j}\right)}$$

---
$$
\begin{align}
Q_{j}\left(z_{j}\right)&=P\left(z_{j} \mid x_{ji}, \theta_{t}\right)\\
&=\frac{P\left(x_{ji}, z_{j} ; \theta_t\right)}{\sum_z P\left(x_{ji}, z_{j} ; \theta_t\right)}【这个计算关系上文中有】\\
&=\frac{P(x_{ji}|z_j;\theta_t)P(z_j;\theta_t)}{\sum_z P(x_{ji}|z_j;\theta_t)P(z_j;\theta_t)}
\end{align}
$$

因为$z_j=\{0,1\}$，所以$\sum_{z_j}Q(z_j)=Q(0)+Q(1)$，所以我们要计算出$Q(0)和Q(1)$。


$$Q_{j}\left(1\right)=\frac{P(x_{ji}|1;\theta_t)P(1;\theta_t)}{\sum_z P(x_{ji}|z_j;\theta_t)P(z_j;\theta_t)}$$

$$
\begin{align}
因为：P(1;\theta_t)&=\pi_t \\
P(x_{ji}|1;\theta_t)&=\prod_{i=1}^n p_t^{x_{ji}} (1-p_t)^{1-x_{ji}} \\
P(0;\theta_t)&=1-\pi_t \\
P(x_{ji}|0;\theta_t)&=\prod_{i=1}^n q_t^{x_{ji}} (1-q_t)^{1-x_{ji}} \\
所以：Q_{j}\left(1\right)&=\frac{\pi_t\prod_{i=1}^n(p_t^{x_{ji}} (1-p_t)^{1-x_{ji}})}{\pi_t\prod_{i=1}^n(p_t^{x_{ji}} (1-p_t)^{1-x_{ji}})+(1-\pi_t)\prod_{i=1}^n(q_t^{x_{ji}} (1-q_t)^{1-x_{ji}})} \\
Q_{j}\left(0\right)&=1-Q_{j}\left(1\right) \\
\end{align}
$$

所以：
$$l(\theta,\theta_t)=\sum_{j=1}^m\sum_{i=1}^n\left[Q_j(1)\log \frac{\pi\prod_{i=1}^n (p^{x_{ji}}(1-p)^{1-x_{ji}})}{Q_j(1)} + Q_j(0)\log \frac{(1-\pi)\prod_{i=1}^n (q^{x_{ji}}(1-q)^{1-x_{ji}})}{Q_j(0)}\right]$$




求每个变量的偏导：
$$
\begin{align}
\frac{\partial f}{\partial \pi}&=\sum_{j=1}^m\sum_{i=1}^{n} \frac{Q_j(1)-\pi}{\pi(1-\pi)} =n(\frac{Q_1(1)-\pi}{\pi(1-\pi)})+n(\frac{Q_2(1)-\pi}{\pi(1-\pi)})+...+n(\frac{Q_m(1)-\pi}{\pi(1-\pi)})=n(\frac{\sum_{j=1}^m Q_j(1)-m\pi}{\pi(1-\pi)})= 0 \\
\frac{\partial f}{\partial p}&=\sum_{j=1}^m\sum_{i=1}^{n} \frac{x_{ji}-p}{p(1-p)} Q_j(1) = 0 \\
\frac{\partial f}{\partial q}&=\sum_{j=1}^m\sum_{i=1}^{n} \frac{x_{ji}-q}{q(1-q)}\left(1-Q_j(1)\right) = 0
\end{align}
$$


令偏导数为0，求得下一次变量的值：
$$
\begin{align} 
\pi &=\frac{1}{m} \sum_{j=1}^m Q_j(1) \\ 
p &=\frac{\sum_{j=1}^m\sum_{i=1}^{n} Q_j(1) x_{ji}}{\sum_{j=1}^m\sum_{i=1}^{n} Q_j(1)} \\ 
q &=\frac{\sum_{j=1}^m \sum_{i=1}^{n}\left(1-Q_j(1)\right) x_{ji}}{\sum_{j=1}^m \sum_{i=1}^{n}\left(1-Q_j(1)\right)}\\
\end
{align}$$

In [29]:
# -*- coding: utf-8 -*-
# @author : wanglei
# @date : 2021/4/16 3:58 PM
# @description :


import numpy as np

y = np.array([[1, 0, 0, 0, 1, 1, 0, 1, 0, 1],
              [1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
              [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
              [1, 0, 1, 0, 0, 0, 1, 1, 0, 0],
              [0, 1, 1, 1, 0, 1, 1, 1, 0, 1]])
M, N = y.shape
print(M, N)  # 5，10
pi_n = 0.5
p_n = 0.5
q_n = 0.6
flag = 1
iter = 0
while flag:
    pi = pi_n
    p = p_n
    q = q_n
    Q = np.zeros(M)
    for j in range(M):
        s1 = 1
        s2 = 1
        for i in range(N):
            s1 *= np.power(p, y[j][i]) * np.power(1 - p, 1 - y[j][i])
            s2 *= np.power(q, y[j][i]) * np.power(1 - q, 1 - y[j][i])
        t1 = pi * s1
        t2 = (1 - pi) * s2
        Q[j] = t1 / (t1 + t2)

    pi_n = 0.5 #np.sum(Q)/M
    p_n = np.sum([np.sum(y[j] * Q[j]) for j in range(M)]) / np.sum(N*Q)
    q_n = np.sum([np.sum(y[j] * (1 - Q[j])) for j in range(M)]) / np.sum(N*(1-Q))
    print(('%1.4f %5.4f %5.4f') % (pi_n, p_n, q_n))
    iter = iter + 1
    if iter == 10:
        flag = 0

5 10
0.5000 0.5813 0.7130
0.5000 0.5693 0.7453
0.5000 0.5495 0.7681
0.5000 0.5346 0.7832
0.5000 0.5263 0.7911
0.5000 0.5224 0.7945
0.5000 0.5207 0.7959
0.5000 0.5200 0.7965
0.5000 0.5198 0.7967
0.5000 0.5197 0.7967


In [23]:
from sympy import *

x = symbols("x")# 符号x，自变量
y = ln(1-x) # 公式

dify = diff(y,x) #求导
print(dify)

-1/(1 - x)
